<a href="https://colab.research.google.com/github/a-anksri/sculpture/blob/main/annotation_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import PIL
import pandas as pd
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
landmarks = ["Root", "Forehead", "Left Eye", "Left Ear", "Left Shoulder", "Left Waist", "Right Eye", "Right Ear", "Right Shoulder", "Right Waist", "Nose", "Left Elbow", "Left Knee", "Right Elbow", "Right Knee", "Left Wrist", "Left Foot", "Right Wrist", "Right Foot"]
limbs = {"Root":[1], "Forehead":[10,2,3,4,6,7,8], "Left Shoulder":[11,5], "Left Elbow": [15], "Left Waist":[12], "Left Knee": [16], "Right Shoulder":[13,9], "Right Elbow": [17], "Right Waist":[14], "Right Knee": [18]}
possible_duplicates = [1,4,8,11,12,13,14]


In [ ]:
class Link:

  def __init__(self, landmark, possible = False):
    self.landmark_type = landmark
    self.parent = self
    self.children = []
    self.next_child = 0
    self.duplicate_possible = possible
  
  def add_parent(self, parent):
    self.parent = parent

  def add_child(self, child):
    self.children.append(child)

  def get_type(self):
    return self.landmark_type

  def get_parent(self):
    return self.parent

  def check_over(self):
    if(self.next_child == len(self.children)):
      
      return(True)
    else:
      return(False)
  
  

class Chain:

  def __init__(self, landmarks, limbs, possible_duplicates):
    self.landmarks = landmarks
    self.limbs = limbs
    self.possible_duplicates = possible_duplicates
    self.links = {}
    for i, landmark in enumerate(self.landmarks):
      if(i in self.possible_duplicates):
        link = Link(landmark, True)
      else:
        link = Link(landmark, False)
      self.links[landmark] = link

    self.links["Root"].add_parent(None)

    for limb in limbs:
      parent = self.links[limb]
      for i in limbs[limb]:
        self.links[self.landmarks[i]].add_parent(parent)
        self.links[parent.get_type()].add_child(self.links[self.landmarks[i]])

  def get_root(self):
    return self.links["Root"]
  
  


  def traverse(self):
      next = self.links["Root"]
      while(next is not None):
        print(next.get_type())
        next, _ = next.successor()
        



    

    

In [ ]:
class Annotation:

  def __init__(self, landmarks, limbs):
    self.tree = {"id":[],"pid":[], "type": [], "x":[], "y": [], "attr":[]}
    self.count = 0
    self.landmarks = landmarks
    self.limbs = limbs
    self.chain = None
    self.roots = []
    self.current_root = None
    self.current_id = None
    self.current_link = None
    self.img = None
    self.pane = np.zeros((256,256, 3), np.uint8)
    self.temp_pane = None
    self.menu = None
    self.menuText = ''
    self.temp_entry = {}
    self.elements = {"yes":[], "no":[]}

    self.state = 'main'

  def load_image(self, image, image_name = ''):

    self.img = image

    self.menuText = 'Loaded Image ' + image_name
    self.pane = self.img.copy()
    self.state = 'main'
    

  def get_state(self):
    return(self.state)
  


  
#Initialising functions
  def add_root(self, attr = ''):

    id = self.count
    self.tree['id'] = id
    self.tree['pid'] = -1
    self.tree['type'] = "Root"
    self.tree['x'] = 0
    self.tree['y'] = 0
    self.tree['attr'] = attr
    self.current_id = id
    self.count += 1
    self.state = 'select'
    return(id)
  
  def start_annotation(self):
    if(self.state != 'main'):
      return(-1)

    self.chain = Chain(landmarks, limbs)
    self.current_link = self.chain.get_root()
    self.current_id = self.add_root()
    self.current_root = self.current_id
    self.roots.append(id)
    self.current_link, _ = self.current_link.successor()
    return(0)

#Entry functions

  def capture(self, x,y):
    if(self.state == 'select'):
      
      l_type = self.current_link.get_type()
      self.temp_add(l_type, x, y, '')
      self.draw_point_on_pane(x,y)
      self.state = 'confirm'
  
  def temp_add(self, l_type, x, y, attr = ''):
    self.temp_entry["type"] = l_type
    self.temp_entry['x'] = x
    self.temp_entry['y'] = y
    self.temp_entry['attr'] = attr


  def add_entry(self):

    id = self.count
    self.tree['id'] = id
    self.tree['pid'] = self.current_id
    self.tree['type'] = self.temp_entry["type"]
    self.tree['x'] = self.temp_entry['x']
    self.tree['y'] = self.temp_entry['y']
    self.tree['attr'] = self.temp_entry['attr']
    self.current_id = id
    self.count += 1
    return(id)


#GUI functions
  def draw_point_on_pane(self, x, y):
    self.temp_pane = self.pane.copy()
    self.pane = cv2.circle(self.pane, (x,y), 3,(255,0,0), -1)

  def draw_confirmation(self,x,y):
    lx1 = x -60
    lx2 = x + 10
    ly = y + 50
    self.temp_pane = self.pane.copy()
    self.pane = cv2.circle(self.pane, (lx1,ly), 22, (0,255,0), -1)
    self.pane = cv2.circle(self.pane, (lx1,ly), 25, (255,255,255), 3)
    self.pane = cv2.circle(self.pane, (lx2,ly), 22, (0,0,255), -1)
    self.pane = cv2.circle(self.pane, (lx2,ly), 25, (255,255,255), 3)

    self.pane = cv2.putText(self.pane, "Yes")
    #code
    pass
  
  def refresh_pane(self):
    #code
    pass
  
  def within(x,y):

    for i in self.elements:
      if(((x-i[0])**2 + (y - i[1])**2) < 2**2):
        return(i[2])


  
    
  
  
      

#Logic for Annotation Progression
  def do_confirm(self, i):

    if(i == 0):
      self.add_entry()
      self.temp_entry = None
      self.refresh_pane(self.current_id)

      next, over, par = self.current_link.successor()

      if(over):
        self.state = 'select'
        if(par):
        self.current_id = self.tree['pid'][self.current_id]
        self.menuText('Select Another ' + next.get_type() + ' connected to highlighted. Press n to go up')
        self.current_link = next

      else:
        self.state = 'select'
        self.menuText = ("Select a " + next.get_type() + 'connected to highlighted landmark. Press n to go up')
        self.current_link = next
    
    else:
      self.temp_entry = None
      self.refresh_pane(self.current_id)

  def do_select(self, x, y):
    if(self.state == 'select'):
      self.capture(x,y) 
      self.draw_confirmation(x,y)   

    if(self.state == 'confirm'):
      response = self.within(x,y)
      do_confirm(response)

  def go_up(self):
    if(state == 'select'):
      next, over = self.current_link.successor(up = True)

      if(over):
        self.state = 'select'
        self.current_id = self.tree['pid'][self.current_id]
        self.menuText('Select Another ' + next.get_type() + ' connected to highlighted. Press n to go up')
        self.current_link = next
      
      else:
        self.state = 'select'
        self.menuText = ("Select a " + next.get_type() + 'connected to highlighted landmark. Press n to go up')
        self.current_link = next







IndentationError: ignored

In [ ]:
class Final_Annotation():

  def __init__(self):
    self.annotations = {"id":[],"pid":[], "type": [], "x":[], "y": [], "attr":[], "person":[]}
    self.next_id = 0
    self.roots = []

  def append(self,annot):
    for col in self.annotations:
      self.annotations[col] = self.annotations[col] + annot.tree[col]
    self.next_id += annot.count
    self.roots = self.roots + annot.roots
    return(self.next_id)

In [ ]:

class Annotation_noGUI:

  def __init__(self, landmarks, limbs, possible_duplicates, person_id = -1):
    self.tree = {"id":[],"pid":[], "type": [], "x":[], "y": [], "attr":[], "person":[]}
    self.count = 0
    self.landmarks = landmarks
    self.limbs = limbs
    self.possible_duplicates = possible_duplicates
    self.chain = None
    self.roots = []
    self.current_root = None
    self.current_id = None
    self.person_id = person_id
    
    self.next_link = None
    self.parent_link = None
    self.current_parent = None
    self.img = None
    self.pane = np.zeros((256,256, 3), np.uint8)
    self.temp_pane = None
    self.menu = None
    self.menuText = ''
    self.temp_entry = {}
    self.elements = {"yes":[], "no":[]}
    self.over = False

    self.state = 'main'
    

  def get_state(self):
    return(self.state)
  
  def set_state(self, state):
    self.state = state


  

  
  def successor(self, up = False):

    
    if(up):
          if(self.next_link.duplicate_possible):
            self.parent_link.next_child += 1
          else:
            pass

          self.next_link = self.parent_link
          if(self.next_link.get_type() == "Root"):
            return (-1)
          
          self.current_parent = self.tree['pid'][self.current_id]
          self.parent_link = self.next_link.parent
          
          
          self.successor()
          return


    if(self.next_link.next_child < len(self.next_link.children)):
          
          tmp = self.next_link.next_child
          self.parent_link = self.next_link 
          self.current_parent = self.current_id
          self.next_link = self.next_link.children[tmp]
          
          if(self.next_link.duplicate_possible):
            pass
            
          else:
            self.parent_link.next_child += 1
          
          
            
          

    else:
          self.next_link.next_child = 0
          self.next_link = self.parent_link
          
          

          self.current_id = self.current_parent
          
          self.current_parent = self.tree['pid'][self.current_id]
          self.parent_link = self.next_link.parent
          
          self.successor()
    
          
          
    
    
    
    
#Initialising functions
  def add_root(self, attr = ''):

    id = self.count
    self.tree['id'].append(id)
    self.tree['pid'].append(-1)
    self.tree['type'].append("Root")
    self.tree['x'].append(0)
    self.tree['y'].append(0)
    self.tree['attr'].append(attr)
    self.tree['person'].append(self.person_id)
    self.current_id = id
    self.count += 1
    self.state = 'select'
    self.current_parent = id
    self.parent_link = self.chain.get_root()
    self.next_link = self.parent_link
    
    return(id)
  
  def start_annotation(self):
    if(self.state != 'main'):
      return(-1)
    
    self.chain = Chain(landmarks, limbs, possible_duplicates)
    self.current_id = self.add_root()
    self.current_root = self.current_id
    self.roots.append(self.current_id)
    return(0)

#Entry functions

  def capture(self, x,y):
    if(self.state == 'select'):
      
      l_type = self.next_link.get_type()
      self.temp_add(l_type, x, y, '')
      #self.draw_point_on_pane(x,y)
  
  def temp_add(self, l_type, x, y, attr = ''):
    self.temp_entry["type"] = l_type
    self.temp_entry['x'] = x
    self.temp_entry['y'] = y
    self.temp_entry['attr'] = attr
    


  def add_entry(self):

    id = self.count
    self.tree['id'].append(id)
    self.tree['pid'].append(self.current_parent)
    self.tree['type'].append(self.temp_entry["type"])
    self.tree['x'].append(self.temp_entry['x'])
    self.tree['y'].append(self.temp_entry['y'])
    self.tree['attr'].append(self.temp_entry['attr'])
    self.tree['person'].append(self.person_id)
    self.current_id = id
    print(self.tree['type'][self.current_id] + " Added with id {}".format(self.current_id))

    self.count += 1
    out = self.successor()
    
    return(id)




#Logic for Annotation Progression
  def do_confirm(self, i):

    if(self.next_link.get_type() == 'Root'):
        self.successor()
        self.state = 'select'
        print('Select A(An) ' + self.next_link.get_type() + ' For the person')
        return(0)

    
    if(i == 0):
      
      self.add_entry()
      
      self.temp_entry = {}
      #self.refresh_pane(self.current_id)
      




      
      if(self.next_link.get_type() == 'Root'):
        return(-1)

      
      self.state = 'select'
      print('Select A(An) ' + self.next_link.get_type() + ' connected to previously selected ' + self.parent_link.get_type() + " {}. Press n to go up".format(self.tree['id'][self.current_parent]))
        
    
    else:
      self.temp_entry = {}
      self.state = 'select'
      print('Select again')
      #self.refresh_pane(self.current_id)
    
    
    return(0)


  def do_select(self, x, y):
    if(self.state == 'select'):
      self.capture(x,y)
      
      self.state = 'confirm' 
      #self.draw_confirmation(x,y)   

    

  def go_up(self):
    
      
      out = self.successor(up = True)
      if(self.next_link.get_type() == 'Root'):
        return(-1)
      
      
      self.state = 'select'
      print('Select A(An) ' + self.next_link.get_type() + ' connected to previously selected ' + self.parent_link.get_type() + " {}. Press n to go up".format(self.tree['id'][self.current_parent]))



In [ ]:
#noGUI



def tool_noGUI(img_name, All_annotations):
  
  person_id = 0
  
  to_save = True
  while(True):
    annot = Annotation_noGUI(landmarks, limbs, possible_duplicates, person_id = person_id)
    annot.start_annotation()
    print('Starting New Person')
    annot.do_confirm(0)

    
    
    while(True):

      state = annot.get_state()
      
      
      if(state == 'select'):
            
            ok = input("Type y to show selection. n to go up. q to quit current person")
            if(ok == 'y'):
              
              annot.do_select(0,0)
            elif(ok == 'q'):
              save = input("Annotation of current person may be incomplete. Want to save?")
              if(save == 'y'):
                to_save = True
              else:
                to_save = False
              print("Quitting annotation of current person")
              break
            elif(ok == 'n'):
              
              res = annot.go_up()
              if(res == -1):
                break
            else:
              print("wrong selection")
            

      elif(state == 'confirm'):
          ok = input("Confirm selection ")
          if(ok == 'y'):

            res = annot.do_confirm(0)
          else:
            res = annot.do_confirm(1)

          if(res == -1):
            break
        
      elif(state == 'main'):
        pass
    
    if(to_save):
      next_id = All_annotations.append(annot)
      print(annot.tree['id'])

    res = input("more persons in the image? ")
    

    if(res == 'n'):
      break
    else:
      to_save = True
    
    person_id += 1

#file = pd.DataFrame(All_annotations)

    
       


In [ ]:
PATH = ''
All_annotations = Final_Annotation()
tool_noGUI('abc', All_annotations)

Starting New Person
Select A(An) Forehead For the person
Type y to show selection. n to go up. q to quit current persony
Confirm selection y
Forehead Added with id 1
Select A(An) Nose connected to previously selected Forehead 1. Press n to go up
Type y to show selection. n to go up. q to quit current persony
Confirm selection y
Nose Added with id 2
Select A(An) Left Eye connected to previously selected Forehead 1. Press n to go up


KeyboardInterrupt: ignored

ver2
